In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import wandb

import torch
from torch import nn
from torchvision import models
import torch.optim as optim

from dataset import get_dataloaders, get_datasets
from utils import seed_everything
from trainer import Trainer

# Params
Image.MAX_IMAGE_PIXELS = 1e11
CFG = {
    'seed': 42,
    'base_model': 'resnet34',   # resnet18/34/50, efficientnet_v2_s/m/l
    'img_size': 1024,
    'batch_size': 4,
    'freeze_epochs': 1,
    'epochs': 10,
    'base_lr': 1e-3,
    'affine_degrees': 10,
    'affine_translate': (0.1, 0.2),
    'affine_scale': (0.8, 1.2)
}

# Wandb
wandb.login(key='1b0401db7513303bdea77fb070097f9d2850cf3b')
run = wandb.init(project='kaggle-ubc-ocean', config=CFG, tags=['torch', 'baseline'])

# Label encoder/decoder
encode = {'HGSC': 0, 'LGSC': 1, 'EC': 2, 'CC': 3, 'MC': 4}
decode = {v: k for k, v in encode.items()}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Paths
root = '/media/latlab/MR/projects/kaggle-ubc-ocean'
data_dir = os.path.join(root, 'data')
results_dir = os.path.join(root, 'results')
train_csv = 'train.csv'
train_image_dir = os.path.join(data_dir, 'train_images')
train_thumbnail_dir = os.path.join(data_dir, 'train_thumbnails')

# Seed
seed_everything(CFG['seed'])

# Load data
df = pd.read_csv(os.path.join(data_dir, train_csv))
df['label'] = df.loc[:,'label'].map(encode)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: naraiadam88. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/latlab/.netrc


In [2]:
def train_model(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation, encode):
    # Data loaders
    datasets = get_datasets(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation)
    dataloaders = get_dataloaders(CFG, datasets)

    # Model definition
    model = models.get_model(CFG['base_model'], weights='DEFAULT').to(device)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, len(encode)).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=CFG['base_lr'], momentum=0.9)
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    # Training
    trainer = Trainer(model, dataloaders, loss_fn, optimizer, exp_lr_scheduler, device, metric='balanced_accuracy', wandb_log=True)
    model, _ = trainer.train_epochs(num_epochs=CFG['freeze_epochs'])
    trainer.unfreeze()
    model, best_balanced_acc = trainer.train_epochs(num_epochs=CFG['epochs'])
    return model, best_balanced_acc

In [3]:
skf = StratifiedKFold(n_splits=2, random_state=CFG['seed'], shuffle=True)
lb = df.label
train_index, valid_index = next(skf.split(np.zeros(len(lb)), lb))
df_train = df.iloc[train_index]
df_validation = df.iloc[valid_index]
model, balanced_acc = train_model(CFG, train_image_dir, train_thumbnail_dir, df_train, df_validation, encode)
torch.save(model.state_dict(), os.path.join(results_dir, 'models', f'ubc-ocean_{run.name}.pt'))
wandb.finish()

Epoch 1/1
----------


100%|██████████| 68/68 [00:08<00:00,  8.43it/s]


train loss: 1.6932, test loss: 1.9282, balanced_accuracy: 0.2463

Training complete in 0m 17s
Final balanced_accuracy: 0.246324

Epoch 1/10
----------


100%|██████████| 68/68 [00:10<00:00,  6.58it/s]


train loss: 1.6243, test loss: 1.5704, balanced_accuracy: 0.4056

Epoch 2/10
----------


...

train loss: 0.7498, test loss: 1.0173, balanced_accuracy: 0.7145

Epoch 10/10
----------


100%|██████████| 68/68 [00:10<00:00,  6.62it/s]


train loss: 0.8027, test loss: 1.0102, balanced_accuracy: 0.6814

Training complete in 3m 29s
Final balanced_accuracy: 0.681373



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


balanced_accuracy,▁▃▆▅▅▇▅▇███
train_loss,█▇▆▄▄▄▅▃▂▁▁
valid_loss,█▅▂▅▃▂▄▂▁▁▁
balanced_accuracy,0.68137
train_loss,0.80274
valid_loss,1.01021
